# Mosaic AI Agent Framework: deploy a tool-calling LangGraph agent


- deploy to production when quality is good enough.





### Load the libraries

In [0]:
%pip install -U -qqqq backoff==2.2.1 databricks-langchain==0.8.2 langgraph==0.6.4 uv databricks-agents==1.9.0 databricks-connect==16.4.7 databricks-langchain==0.8.2 mlflow[databricks]==3.8.1
dbutils.library.restartPython()

### Run Unity Catalog Config

In [0]:
%run ../_config/config_unity_catalog

### Run MLFlow config

In [0]:
%run ../_config/config_agent

## Register the agent 

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow

df_logged_agents=mlflow.search_logged_models()
uri_path = df_logged_agents['artifact_location'].iloc[-1]

In [0]:
loaded_model = mlflow.pyfunc.load_model(uri_path)

## Deploy the agent to production

### Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.


In [0]:
model_name = "langgraph-agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=uri_path, 
    name=UC_MODEL_NAME
)

### Deploy the agent

In [0]:

from databricks import agents

agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    scale_to_zero=True, #Compulsory on the databricks free version
    tags={"endpointSource": "docs"},
)